# Analise de Felicidade - "Somos Felizes?"
**Projeto 1: IBM HR Analytics - AiDAPT - Cegid Academy**

Como novo director de RH, uma das nossas perguntas prioritarias e simples mas fundamental: **os nossos colaboradores sao felizes?**

Para responder, vamos analisar 4 dimensoes de satisfacao, cada uma medida numa escala de 1 (baixo) a 4 (muito alto):

- **EnvironmentSatisfaction** - Satisfacao com o ambiente de trabalho
- **JobSatisfaction** - Satisfacao com o trabalho em si
- **RelationshipSatisfaction** - Satisfacao com as relacoes profissionais
- **WorkLifeBalance** - Equilibrio entre vida pessoal e profissional

Combinando estas 4 dimensoes num **Indice de Felicidade** composto, vamos classificar a empresa como **Feliz** (>= 3.0), **Satisfeita** (2.5 - 3.0) ou **Insatisfeita** (< 2.5).

Base de dados: Projeto1_IBM_HR | 1470 colaboradores

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from urllib.parse import quote_plus
from sqlalchemy import create_engine
load_dotenv(find_dotenv())

%load_ext sql

host = os.getenv('MSSQL_HOST', 'localhost')
port = os.getenv('MSSQL_PORT', '1433')
user = os.getenv('MSSQL_USER', 'sa')
password = quote_plus(os.getenv('MSSQL_PASSWORD', 'your_password_here'))
engine = create_engine(f"mssql+pymssql://{user}:{password}@{host}:{port}/Projeto1_IBM_HR")
%sql engine --alias Projeto1_IBM_HR

Tip: You may define configurations in /Users/diogosilva/Documents/Cegid_Academy/cegid_academy_aidapt/pyproject.toml or /Users/diogosilva/.jupysql/config.

Did not find user configurations in /Users/diogosilva/Documents/Cegid_Academy/cegid_academy_aidapt/pyproject.toml.

## LEGENDA DAS ESCALAS DE SATISFAÇÃO

**Legenda:**
```
EnvironmentSatisfaction, JobSatisfaction, RelationshipSatisfaction, JobInvolvement:
    1 = Low (Baixo)
    2 = Medium (Médio)
    3 = High (Alto)
    4 = Very High (Muito Alto)

WorkLifeBalance:
    1 = Bad (Mau)
    2 = Good (Bom)
    3 = Better (Melhor)
    4 = Best (Excelente)
```

## 1. A RESPOSTA: SOMOS FELIZES?

Comecemos pelo resultado principal. Antes de mergulhar nos detalhes, vejamos como se posicionam as 5 dimensoes de satisfacao da empresa.

### Medias globais de satisfacao (escala 1-4)

In [2]:
%%sql
SELECT
    ROUND(AVG(CAST(EnvironmentSatisfaction AS FLOAT)), 2) AS MediaSatisfacaoAmbiente,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS MediaSatisfacaoTrabalho,
    ROUND(AVG(CAST(RelationshipSatisfaction AS FLOAT)), 2) AS MediaSatisfacaoRelacoes,
    ROUND(AVG(CAST(WorkLifeBalance AS FLOAT)), 2) AS MediaWorkLifeBalance,
    ROUND(AVG(CAST(JobInvolvement AS FLOAT)), 2) AS MediaEnvolvimento
FROM Colaboradores;

Running query in 'Projeto1_IBM_HR'

MediaSatisfacaoAmbiente,MediaSatisfacaoTrabalho,MediaSatisfacaoRelacoes,MediaWorkLifeBalance,MediaEnvolvimento
2.72,2.73,2.71,2.76,2.73


### Indice de Felicidade composto

Combinando as 4 dimensoes principais (excluindo JobInvolvement, que mede envolvimento e nao satisfacao) num unico numero:

In [3]:
%%sql
SELECT
    ROUND(AVG(
        (CAST(EnvironmentSatisfaction AS FLOAT) +
         CAST(JobSatisfaction AS FLOAT) +
         CAST(RelationshipSatisfaction AS FLOAT) +
         CAST(WorkLifeBalance AS FLOAT)) / 4.0
    ), 2) AS IndiceFelicidadeGlobal,
    CASE
        WHEN AVG((EnvironmentSatisfaction + JobSatisfaction + RelationshipSatisfaction + WorkLifeBalance) / 4.0) >= 3 THEN 'FELIZES'
        WHEN AVG((EnvironmentSatisfaction + JobSatisfaction + RelationshipSatisfaction + WorkLifeBalance) / 4.0) >= 2.5 THEN 'SATISFEITOS'
        ELSE 'INSATISFEITOS'
    END AS Classificacao
FROM Colaboradores;

Running query in 'Projeto1_IBM_HR'

IndiceFelicidadeGlobal,Classificacao
2.73,SATISFEITOS


**Resultado: Indice de Felicidade = 2.73/4 - SATISFEITOS**

Todos os indicadores estao muito proximos (entre 2.71 e 2.76), e todos abaixo de 3.0 ("High"). A empresa esta no limiar entre satisfeita e feliz - **nao ha alarme, mas ha espaco claro para melhoria**.

- **Melhor indicador**: WorkLifeBalance (2.76) - o equilibrio vida-trabalho e onde nos saimos melhor
- **Pior indicador**: RelationshipSatisfaction (2.71) - as relacoes profissionais sao o ponto mais fraco

Mas o que significa este 2.73 na pratica? Quantos colaboradores estao realmente satisfeitos e quantos nao estao? Vamos olhar para as distribuicoes.

## 2. OLHAR PARA DENTRO DE CADA DIMENSAO

O indice global de 2.73 esconde realidades diferentes. Vejamos como se distribuem os colaboradores em cada dimensao - quantos estao realmente satisfeitos e quantos nao estao?

### Satisfacao com o Ambiente (EnvironmentSatisfaction)

**61.2% com satisfacao alta ou muito alta** (niveis 3-4). Parece positivo, mas o reverso e preocupante: **19.3% tem satisfacao baixa** com o ambiente - quase 1 em cada 5 colaboradores. Os dois extremos (Low e Very High) tem proporcoes quase identicas (~19% e ~30%), o que sugere uma polarizacao.

In [4]:
%%sql
SELECT
    EnvironmentSatisfaction AS Nivel,
    CASE EnvironmentSatisfaction
        WHEN 1 THEN 'Low'
        WHEN 2 THEN 'Medium'
        WHEN 3 THEN 'High'
        WHEN 4 THEN 'Very High'
    END AS Descricao,
    COUNT(*) AS Total,
    CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS Percentagem
FROM Colaboradores
GROUP BY EnvironmentSatisfaction
ORDER BY EnvironmentSatisfaction;

Running query in 'Projeto1_IBM_HR'

Nivel,Descricao,Total,Percentagem
1,Low,284,19.3
2,Medium,287,19.5
3,High,453,30.8
4,Very High,446,30.3


**61.3% satisfeitos** com o trabalho (niveis 3-4). Padrao muito similar ao ambiente: cerca de 20% em cada nivel, com ligeira concentracao nos niveis altos. **19.7% com satisfacao baixa** - novamente, quase 1 em 5.

### Satisfacao com o Trabalho (JobSatisfaction)

**WorkLifeBalance e o nosso melhor indicador**: 71.2% nos niveis "Better" ou "Best" (3-4). A grande maioria (60.7%) concentra-se no nivel "Better". Apenas **5.4% reporta WLB "Bad"** - um grupo pequeno (80 pessoas) mas critico, que merece atencao.

A diferenca em relacao ao ambiente e ao trabalho e notavel: enquanto esses tinham ~20% de insatisfeitos, aqui sao apenas 5.4%.

### Sintese das distribuicoes

| Dimensao | % Satisfeitos (3-4) | % Insatisfeitos (1) |
|----------|---------------------|---------------------|
| WorkLifeBalance | **71.2%** | 5.4% |
| JobSatisfaction | 61.3% | 19.7% |
| EnvironmentSatisfaction | 61.2% | 19.3% |

**Padrao claro**: a maioria esta satisfeita (61-71%), mas existe um nucleo de ~20% insatisfeitos com o ambiente e o trabalho. O WLB destaca-se pela positiva com poucos insatisfeitos. A preocupacao principal sao os quase **1 em cada 5** colaboradores com satisfacao baixa no ambiente e no trabalho.

**Surpresa**: HR e o departamento mais feliz (2.77), nao o mais infeliz. Mas as **diferencas sao minimas** - apenas 0.04 pontos entre o melhor e o pior departamento.

HR destaca-se em Relacoes (2.89) e WorkLife (2.92), mas tem a **pior satisfacao com o Trabalho** (2.60). R&D, o maior departamento com 961 pessoas, tem o indice mais baixo (2.73) mas sem ser alarmante.

**Conclusao**: a felicidade e uniforme entre departamentos - nao ha um departamento problematico.

In [5]:
%%sql
SELECT
    JobSatisfaction AS Nivel,
    CASE JobSatisfaction
        WHEN 1 THEN 'Low'
        WHEN 2 THEN 'Medium'
        WHEN 3 THEN 'High'
        WHEN 4 THEN 'Very High'
    END AS Descricao,
    COUNT(*) AS Total,
    CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS Percentagem
FROM Colaboradores
GROUP BY JobSatisfaction
ORDER BY JobSatisfaction;

Running query in 'Projeto1_IBM_HR'

Nivel,Descricao,Total,Percentagem
1,Low,289,19.7
2,Medium,280,19.0
3,High,442,30.1
4,Very High,459,31.2


Manufacturing Director lidera (2.77) e Research Director e o menos feliz (2.69). Mas a amplitude e de apenas **0.08 pontos** - nao ha um cargo dramaticamente mais infeliz que os outros.

**Nota**: Research Director tem o pior indice apesar de ser um cargo senior. Lab Technician (2.70) e o segundo pior, sendo simultaneamente um dos cargos com maior attrition (analise aprofundada no notebook 07).

### Equilibrio Vida-Trabalho (WorkLifeBalance)

In [6]:
%%sql
SELECT
    WorkLifeBalance AS Nivel,
    CASE WorkLifeBalance
        WHEN 1 THEN 'Bad'
        WHEN 2 THEN 'Good'
        WHEN 3 THEN 'Better'
        WHEN 4 THEN 'Best'
    END AS Descricao,
    COUNT(*) AS Total,
    CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS Percentagem
FROM Colaboradores
GROUP BY WorkLifeBalance
ORDER BY WorkLifeBalance;

Running query in 'Projeto1_IBM_HR'

Nivel,Descricao,Total,Percentagem
1,Bad,80,5.4
2,Good,344,23.4
3,Better,893,60.7
4,Best,153,10.4


**Resultado contra-intuitivo**: quem faz overtime tem **MAIOR** indice de felicidade (2.79 vs 2.71) e maior satisfacao com o trabalho (2.77 vs 2.71). Apenas o WLB e ligeiramente pior (2.73 vs 2.77).

Possivel explicacao: quem faz overtime pode estar mais envolvido e motivado no seu trabalho, ou recebe compensacao que aumenta a satisfacao. O overtime nao parece ser, por si so, uma fonte de infelicidade - embora, como veremos no notebook de attrition, seja o maior factor de risco para saidas.

## 3. HA DEPARTAMENTOS MAIS FELIZES QUE OUTROS?

Sera que a felicidade varia entre departamentos? Ou estamos todos ao mesmo nivel?

**Impacto minimo**. As diferencas sao de apenas 0.05 pontos. Surpreendentemente, quem viaja frequentemente tem WLB ligeiramente melhor (2.78) do que quem nao viaja (2.77).

**Conclusao**: as viagens de trabalho nao parecem ser um factor significativo de insatisfacao.

In [7]:
%%sql
SELECT
    Department AS Departamento,
    COUNT(*) AS NumColaboradores,
    ROUND(AVG(CAST(EnvironmentSatisfaction AS FLOAT)), 2) AS Ambiente,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS Trabalho,
    ROUND(AVG(CAST(RelationshipSatisfaction AS FLOAT)), 2) AS Relacoes,
    ROUND(AVG(CAST(WorkLifeBalance AS FLOAT)), 2) AS WorkLife,
    ROUND(AVG((CAST(EnvironmentSatisfaction AS FLOAT) + JobSatisfaction + RelationshipSatisfaction + WorkLifeBalance) / 4.0), 2) AS IndiceFelicidade
FROM Colaboradores
GROUP BY Department
ORDER BY IndiceFelicidade DESC;

Running query in 'Projeto1_IBM_HR'

Departamento,NumColaboradores,Ambiente,Trabalho,Relacoes,WorkLife,IndiceFelicidade
Human Resources,63,2.68,2.6,2.89,2.92,2.77
Sales,446,2.68,2.75,2.7,2.82,2.74
Research & Development,961,2.74,2.73,2.71,2.73,2.73


Tendencia ligeira: quem vive mais perto tem satisfacao e WLB um pouco melhores (2.75/2.79 vs 2.74/2.72). Mas as diferencas sao pequenas.

**Conclusao**: a distancia de casa tem impacto marginal na satisfacao.

### E por cargo?

**Aqui ha um sinal claro**: quem nao e promovido ha **5+ anos tem a satisfacao mais baixa** (2.63), significativamente abaixo da media (2.73).

Ponto interessante: o "ponto doce" e 3-5 anos sem promocao (2.82) - nao promovidos recentemente (2.71) nem demasiado tempo atras. Promocoes recentes nao parecem aumentar a satisfacao imediata.

**Conclusao**: a estagnacao na carreira (5+ anos sem promocao) e o unico factor com impacto claro na satisfacao.

In [8]:
%%sql
SELECT
    JobRole AS Cargo,
    COUNT(*) AS NumColaboradores,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS SatisfacaoTrabalho,
    ROUND(AVG(CAST(WorkLifeBalance AS FLOAT)), 2) AS WorkLife,
    ROUND(AVG((CAST(EnvironmentSatisfaction AS FLOAT) + JobSatisfaction + RelationshipSatisfaction + WorkLifeBalance) / 4.0), 2) AS IndiceFelicidade
FROM Colaboradores
GROUP BY JobRole
ORDER BY IndiceFelicidade DESC;

Running query in 'Projeto1_IBM_HR'

Cargo,NumColaboradores,SatisfacaoTrabalho,WorkLife,IndiceFelicidade
Manufacturing Director,145,2.68,2.77,2.77
Manager,102,2.71,2.77,2.76
Human Resources,52,2.56,2.92,2.76
Healthcare Representative,131,2.79,2.7,2.74
Sales Representative,83,2.73,2.89,2.74
Sales Executive,326,2.75,2.8,2.73
Research Scientist,292,2.77,2.68,2.72
Laboratory Technician,259,2.69,2.72,2.7
Research Director,80,2.7,2.86,2.69


**Surpresa**: o salario **nao correlaciona** com a felicidade. A faixa mais baixa (<$3,000) tem indice identico a faixa mais alta (>$10,000): ambos 2.75. A faixa media (3000-6000) tem o indice mais baixo (2.70).

**Conclusao**: o dinheiro nao compra felicidade nesta empresa. A satisfacao e impulsionada por outros factores - progressao na carreira, envolvimento no trabalho, relacoes profissionais.

---

### Resumo dos factores testados

| Factor | Impacto na felicidade |
|--------|----------------------|
| Overtime | Contra-intuitivo: mais felizes |
| Viagens de trabalho | Sem impacto |
| Distancia de casa | Marginal |
| **Anos sem promocao** | **Unico factor significativo** (5+ anos = queda) |
| Salario | Sem correlacao |

**O resultado mais importante desta seccao**: dos 5 factores testados, apenas a estagnacao na carreira (5+ anos sem promocao) tem impacto claro na satisfacao.

## 4. O QUE INFLUENCIA A FELICIDADE?

Sabemos que somos "satisfeitos" e que as diferencas entre departamentos sao minimas. Mas o que realmente afecta a felicidade dos nossos colaboradores? Vamos testar varios factores.

### 4.1 As horas extra fazem-nos menos felizes?

In [9]:
%%sql
SELECT
    OverTime,
    COUNT(*) AS NumColaboradores,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS SatisfacaoMedia,
    ROUND(AVG(CAST(WorkLifeBalance AS FLOAT)), 2) AS WorkLifeMedia,
    ROUND(AVG((CAST(EnvironmentSatisfaction AS FLOAT) + JobSatisfaction + RelationshipSatisfaction + WorkLifeBalance) / 4.0), 2) AS IndiceFelicidade
FROM Colaboradores
GROUP BY OverTime;

Running query in 'Projeto1_IBM_HR'

OverTime,NumColaboradores,SatisfacaoMedia,WorkLifeMedia,IndiceFelicidade
Yes,416,2.77,2.73,2.79
No,1054,2.71,2.77,2.71


### 4.2 As viagens de trabalho afectam a satisfacao?

In [10]:
%%sql
SELECT
    BusinessTravel AS TipoViagem,
    COUNT(*) AS NumColaboradores,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS SatisfacaoMedia,
    ROUND(AVG(CAST(WorkLifeBalance AS FLOAT)), 2) AS WorkLifeMedia,
    ROUND(AVG((CAST(EnvironmentSatisfaction AS FLOAT) + JobSatisfaction + RelationshipSatisfaction + WorkLifeBalance) / 4.0), 2) AS IndiceFelicidade
FROM Colaboradores
GROUP BY BusinessTravel
ORDER BY IndiceFelicidade DESC;

Running query in 'Projeto1_IBM_HR'

TipoViagem,NumColaboradores,SatisfacaoMedia,WorkLifeMedia,IndiceFelicidade
Non-Travel,150,2.79,2.77,2.77
Travel_Frequently,277,2.79,2.78,2.76
Travel_Rarely,1043,2.7,2.76,2.72


### 4.3 Viver longe da empresa importa?

In [11]:
%%sql
SELECT
    CASE
        WHEN DistanceFromHome <= 5 THEN '0-5 (Muito Perto)'
        WHEN DistanceFromHome <= 10 THEN '6-10 (Perto)'
        WHEN DistanceFromHome <= 15 THEN '11-15 (Médio)'
        ELSE '16+ (Longe)'
    END AS DistanciaCategoria,
    COUNT(*) AS NumColaboradores,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS SatisfacaoMedia,
    ROUND(AVG(CAST(WorkLifeBalance AS FLOAT)), 2) AS WorkLifeMedia
FROM Colaboradores
GROUP BY
    CASE
        WHEN DistanceFromHome <= 5 THEN '0-5 (Muito Perto)'
        WHEN DistanceFromHome <= 10 THEN '6-10 (Perto)'
        WHEN DistanceFromHome <= 15 THEN '11-15 (Médio)'
        ELSE '16+ (Longe)'
    END
ORDER BY MIN(DistanceFromHome);

Running query in 'Projeto1_IBM_HR'

DistanciaCategoria,NumColaboradores,SatisfacaoMedia,WorkLifeMedia
0-5 (Muito Perto),632,2.75,2.79
6-10 (Perto),394,2.7,2.75
11-15 (Médio),115,2.66,2.76
16+ (Longe),329,2.74,2.72


**Este e o resultado mais importante de toda a analise de felicidade.**

Quem saiu da empresa tinha um indice de felicidade de **2.55** vs **2.77** de quem ficou - uma diferenca de 0.22 pontos. A satisfacao com o trabalho mostra o maior gap: **2.47 vs 2.78**.

**A felicidade importa**: esta directamente ligada a retencao. Colaboradores menos felizes saem mais. Isto valida a importancia de monitorizar os indicadores de satisfacao como ferramenta de gestao de retencao.

### 4.4 Ser promovido faz diferenca?

In [12]:
%%sql
SELECT
    CASE
        WHEN YearsSinceLastPromotion = 0 THEN '0 (Promovido recentemente)'
        WHEN YearsSinceLastPromotion <= 2 THEN '1-2 anos'
        WHEN YearsSinceLastPromotion <= 5 THEN '3-5 anos'
        ELSE '5+ anos'
    END AS AnosSemPromocao,
    COUNT(*) AS NumColaboradores,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS SatisfacaoMedia
FROM Colaboradores
GROUP BY
    CASE
        WHEN YearsSinceLastPromotion = 0 THEN '0 (Promovido recentemente)'
        WHEN YearsSinceLastPromotion <= 2 THEN '1-2 anos'
        WHEN YearsSinceLastPromotion <= 5 THEN '3-5 anos'
        ELSE '5+ anos'
    END
ORDER BY MIN(YearsSinceLastPromotion);

Running query in 'Projeto1_IBM_HR'

AnosSemPromocao,NumColaboradores,SatisfacaoMedia
0 (Promovido recentemente),581,2.71
1-2 anos,516,2.76
3-5 anos,158,2.82
5+ anos,215,2.63


### 4.5 E o salario?

In [13]:
%%sql
SELECT
    CASE
        WHEN MonthlyIncome < 3000 THEN 'Baixo (<3000)'
        WHEN MonthlyIncome < 6000 THEN 'Médio (3000-6000)'
        WHEN MonthlyIncome < 10000 THEN 'Alto (6000-10000)'
        ELSE 'Muito Alto (>10000)'
    END AS FaixaSalarial,
    COUNT(*) AS NumColaboradores,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS SatisfacaoMedia,
    ROUND(AVG((CAST(EnvironmentSatisfaction AS FLOAT) + JobSatisfaction + RelationshipSatisfaction + WorkLifeBalance) / 4.0), 2) AS IndiceFelicidade
FROM Colaboradores
GROUP BY
    CASE
        WHEN MonthlyIncome < 3000 THEN 'Baixo (<3000)'
        WHEN MonthlyIncome < 6000 THEN 'Médio (3000-6000)'
        WHEN MonthlyIncome < 10000 THEN 'Alto (6000-10000)'
        ELSE 'Muito Alto (>10000)'
    END
ORDER BY MIN(MonthlyIncome);

Running query in 'Projeto1_IBM_HR'

FaixaSalarial,NumColaboradores,SatisfacaoMedia,IndiceFelicidade
Baixo (<3000),395,2.75,2.75
Médio (3000-6000),519,2.7,2.7
Alto (6000-10000),275,2.76,2.74
Muito Alto (>10000),281,2.71,2.75


## 5. A FELICIDADE IMPORTA? O ELO COM A RETENCAO

Ate agora vimos que somos "satisfeitos" e explorámos os factores que influenciam a felicidade. Mas sera que a felicidade tem impacto real no negocio? Vejamos se quem saiu da empresa era mais ou menos feliz.

A maioria esta bem: **61-71% com satisfacao alta** conforme a dimensao. Mas o reverso tambem e verdade: **29-39% nao estao nos niveis altos**. O WLB e onde estamos melhor (71.2% satisfeitos), o ambiente e trabalho ficam nos ~61%.

### Quem saiu era mais ou menos feliz?

Tres grupos a monitorizar:

1. **28.3% com overtime** (416 colaboradores) - apesar de reportarem mais felicidade, o overtime e o maior factor de risco para saidas (analise no notebook 07)
2. **19.7% com satisfacao baixa no trabalho** (289 colaboradores) - quase 1 em cada 5
3. **5.4% com WLB "Bad"** (80 colaboradores) - grupo pequeno mas com a maior taxa de attrition (31.3%)

---

In [16]:
%%sql
SELECT
    Attrition,
    COUNT(*) AS Total,
    ROUND(AVG(CAST(JobSatisfaction AS FLOAT)), 2) AS SatisfacaoMedia,
    ROUND(AVG(CAST(WorkLifeBalance AS FLOAT)), 2) AS WorkLifeMedia,
    ROUND(AVG((CAST(EnvironmentSatisfaction AS FLOAT) + JobSatisfaction + RelationshipSatisfaction + WorkLifeBalance) / 4.0), 2) AS IndiceFelicidade
FROM Colaboradores
GROUP BY Attrition;

Running query in 'Projeto1_IBM_HR'

Attrition,Total,SatisfacaoMedia,WorkLifeMedia,IndiceFelicidade
Yes,237,2.47,2.66,2.55
No,1233,2.78,2.78,2.77


## 6. SINAIS DE ALERTA: ONDE DEVEMOS INTERVIR?

Sabemos que a felicidade importa para a retencao. Agora, quantos colaboradores estao realmente bem - e quantos nao estao?

### Quantos colaboradores estao realmente bem?

### Percentagem com satisfacao alta ou muito alta

In [19]:
%%sql
SELECT
    'Colaboradores com Satisfação Alta/Muito Alta (3 ou 4)' AS Metrica,
    CAST(ROUND(SUM(CASE WHEN JobSatisfaction >= 3 THEN 1.0 ELSE 0 END) * 100 / COUNT(*), 1) AS VARCHAR(10)) + '%' AS JobSatisfaction,
    CAST(ROUND(SUM(CASE WHEN WorkLifeBalance >= 3 THEN 1.0 ELSE 0 END) * 100 / COUNT(*), 1) AS VARCHAR(10)) + '%' AS WorkLifeBalance,
    CAST(ROUND(SUM(CASE WHEN EnvironmentSatisfaction >= 3 THEN 1.0 ELSE 0 END) * 100 / COUNT(*), 1) AS VARCHAR(10)) + '%' AS EnvironmentSatisfaction
FROM Colaboradores;

Running query in 'Projeto1_IBM_HR'

Metrica,JobSatisfaction,WorkLifeBalance,EnvironmentSatisfaction
Colaboradores com Satisfação Alta/Muito Alta (3 ou 4),61.300000%,71.200000%,61.200000%


### Factores de preocupacao

Quantos colaboradores apresentam factores de risco para a satisfacao?

In [20]:
%%sql
SELECT
    'Colaboradores com Overtime' AS FactorRisco,
    COUNT(*) AS Total,
    CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS Percentagem
FROM Colaboradores
WHERE OverTime = 'Yes'
UNION ALL
SELECT
    'Colaboradores com WorkLife Balance "Bad"',
    COUNT(*),
    CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1))
FROM Colaboradores
WHERE WorkLifeBalance = 1
UNION ALL
SELECT
    'Colaboradores com Job Satisfaction "Low"',
    COUNT(*),
    CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1))
FROM Colaboradores
WHERE JobSatisfaction = 1;

Running query in 'Projeto1_IBM_HR'

FactorRisco,Total,Percentagem
Colaboradores com Overtime,416,28.3
"Colaboradores com WorkLife Balance ""Bad""",80,5.4
"Colaboradores com Job Satisfaction ""Low""",289,19.7
